# Statystyka matematyczna - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu.
- Zadanie należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają za zadanie wyłapać podstawowe błędy. Przejście wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł. 
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem).
- Rozmiar poziomy figur nie powinien przekraczać 20 cali.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu:
Członkowie:

---

# Zestaw zadań 6: Estymacja parametryczna

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów

### Dane do zadań

W celu wygenerowania danych wykorzystywanych w zawartych w notatniku komórkach testowych wykonaj obie poniższe komórki.

In [3]:
# Dane do testów 1
test_data_1 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5, scale=0.2, size=20, random_state=7), columns=["X"])
test_data_1.head()

,X
0,5.338105
1,4.906813
2,5.006564
3,5.081503
4,4.842215


In [4]:
# Dane do testów 2
x = sp.stats.uniform.rvs(loc=-2, scale=10, size=250, random_state=34)
y = 2*x - 5 + sp.stats.norm.rvs(loc=0, scale=2, size=250, random_state=13)
test_data_2 = pd.DataFrame(data=np.array([x, y]).T, columns=["X", "Y"])
test_data_2.head()

,X,Y
0,-1.614383,-9.653548
1,5.801005,8.109542
2,-1.072962,-7.234931
3,4.328927,4.561478
4,-1.861092,-6.031980


### Zadanie 1: Estymacja wartości oczekiwanej [8 pkt]

Przygotuj funkcję `mean_estimation()`, która będzie dokonywała estymacji przedziałowej wartości oczekiwanej dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`𝜇 należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [5]:
def mean_estimation(data, alpha, population_std="unknown"):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i pojedynczą kolumnę "X" z wartościami próby losowej,
    alpha: float
        Wartość określająca poziom istotności.
    population_std: str or float
        Jeżeli odchylenie standardowe populacji nie jest znane to parametr przyjmuje wartość "unknown",
        w przeciwnym wypadku jest to wartość określająca odchylenie standardowe populacji.
    
    Returns
    -------
    mean_low: float
        Dolna granica wyliczonego przedziału ufności.
    mean_high: float
        Górna granica wyliczonego przedziału ufności.
    """
    mean = data.mean().iloc[0]
    length = len(data)
    
    if population_std != "unknown" :
        mi = sp.stats.norm.ppf(1 - alpha / 2)
        temp = mi * population_std / np.sqrt(length)
    else:
        t = sp.stats.t.ppf(1 - alpha / 2, length - 1)
        temp = t * data.values.std() / np.sqrt(length - 1)

    mean_low = mean - temp
    mean_high = mean + temp

    print(f"𝜇 należy do przedziału [{mean_low}, {mean_high}] przy założeniu poziomu ufności 1-{alpha} = {1-alpha}")

    return mean_low, mean_high


In [6]:
### Komórka testowa

# Test na test_data_1 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.1, population_std=0.2), (4.905619719495418, 5.05273990041144))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.05, population_std=0.2), (4.891527555895771, 5.066832064011087))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.02, population_std=0.2), (4.875142370240081, 5.083217249666777))))
print()

# # Test na test_data_1 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.1), (4.913356369231727, 5.045003250675131))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.05), (4.899504007229391, 5.058855612677467))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.02), (4.882508507351398, 5.07585111255546))))

𝜇 należy do przedziału [4.905619719495418, 5.05273990041144] przy założeniu poziomu ufności 1-0.1 = 0.9

𝜇 należy do przedziału [4.891527555895771, 5.066832064011087] przy założeniu poziomu ufności 1-0.05 = 0.95

𝜇 należy do przedziału [4.875142370240081, 5.083217249666777] przy założeniu poziomu ufności 1-0.02 = 0.98

𝜇 należy do przedziału [4.913356369231727, 5.045003250675131] przy założeniu poziomu ufności 1-0.1 = 0.9

𝜇 należy do przedziału [4.899504007229391, 5.058855612677467] przy założeniu poziomu ufności 1-0.05 = 0.95

𝜇 należy do przedziału [4.882508507351398, 5.07585111255546] przy założeniu poziomu ufności 1-0.02 = 0.98


### Zadanie 2: Estymacja wariancji [8 pkt]

Przygotuj funkcję `variance_estimation()`, która będzie dokonywała estymacji przedziałowej wariancji dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`𝜎^2  należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [7]:
def variance_estimation(data, alpha, population_mean="unknown"):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i pojedynczą kolumnę "X" z wartościami próby losowej,
    alpha: float
        Wartość określająca poziom istotności.
    population_mean: str or float
        Jeżeli wartość oczekiwana populacji nie jest znane to parametr przyjmuje wartość "unknown",
        w przeciwnym wypadku jest to wartość określająca wartość oczekiwaną populacji.
    
    Returns
    -------
    var_low: float
        Dolna granica wyliczonego przedziału ufności.
    var_high: float
        Górna granica wyliczonego przedziału ufności.
    """
    length = len(data)
    chi1 = sp.stats.chi2.ppf(1 - alpha / 2, length - 1)
    chi2 = sp.stats.chi2.ppf(alpha / 2, length - 1)

    if population_mean == "unknown":
        population_mean = data.mean().iloc[0]

    S2 = np.sum((data.values - population_mean) **2) / length
    var_low = length * S2 / chi1
    var_high = length * S2 / chi2

    print(f"𝜎^2  należy do przedziału [{var_low}, {var_high}] przy założeniu poziomu ufności 1-𝛼 = {1-alpha}")

    return var_low, var_high

variance_estimation(test_data_1, 0.1, population_mean=5)

𝜎^2  należy do przedziału [0.01855573624556403, 0.05528660848892888] przy założeniu poziomu ufności 1-𝛼 = 0.9


(0.01855573624556403, 0.05528660848892888)

In [8]:
### Komórka testowa

# Test na test_data_1 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.1, population_mean=5), (0.01855573624556403, 0.05528660848892888))));
print()
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.05, population_mean=5), (0.017025745016269588, 0.06280068548350207))));
print()
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.02, population_mean=5), (0.015455150810030724, 0.07328116757245808))));
print()

# Test na test_data_1 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.1), (0.018268125369359626, 0.05442967510189219))));
print()
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.05), (0.0167618487538223, 0.06182728513230527))));
print()
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.02), (0.015215598500840796, 0.07214532146342346))));

𝜎^2  należy do przedziału [0.01855573624556403, 0.05528660848892888] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2  należy do przedziału [0.017025745016269588, 0.06280068548350207] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2  należy do przedziału [0.015455150810030724, 0.07328116757245808] przy założeniu poziomu ufności 1-𝛼 = 0.98

𝜎^2  należy do przedziału [0.018268125369359626, 0.05442967510189219] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2  należy do przedziału [0.0167618487538223, 0.06182728513230527] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2  należy do przedziału [0.015215598500840796, 0.07214532146342346] przy założeniu poziomu ufności 1-𝛼 = 0.98


### Zadanie 3: Estymacja wartości współczynnika korelacji liniowej [6 pkt]

Przygotuj funkcję `correlation_estimation()`, która będzie dokonywała estymacji przedziałowej wartości współczynnika korelacji liniowej Pearsona dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`r należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [9]:
def correlation_estimation(data, alpha):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i dwie kolumny "X" i "Y" z wynikami próby losowej.
    alpha: float
        Wartość określająca poziom istotności.
    Returns
    -------
    r_corr_low: float
        Dolna granica wyliczonego przedziału ufności.
    r_corr_high: float
        Górna granica wyliczonego przedziału ufności.
    """   
    mean_x = data["X"].mean()
    mean_y = data["Y"].mean()
    length = len(data)
    
    r = np.sum((data["X"].values - mean_x) * (data["Y"].values - mean_y)) / (np.sqrt(np.sum((data["X"].values - mean_x) ** 2)) * np.sqrt(np.sum((data["Y"].values - mean_y) ** 2)))
    mi = sp.stats.norm.ppf(1 - alpha/2)
    temp = mi * (1-r ** 2) / np.sqrt(length)
    r_corr_low = r - temp
    r_corr_high = r + temp
    
    print(f"r należy do przedziału [{r_corr_low}, {r_corr_high}] przy założeniu poziomu ufności 1-𝛼 = {1-alpha}")
    
    return r_corr_low, r_corr_high

In [10]:
### Komórka testowa
assert np.all(np.isclose(correlation_estimation(test_data_2, 0.05), (0.9494696249316307, 0.9692185148958979)))
assert np.all(np.isclose(correlation_estimation(test_data_2, 0.02), (0.9476237555239514, 0.9710643843035772)))

r należy do przedziału [0.9494696249316318, 0.9692185148958986] przy założeniu poziomu ufności 1-𝛼 = 0.95
r należy do przedziału [0.9476237555239525, 0.9710643843035779] przy założeniu poziomu ufności 1-𝛼 = 0.98


### Zadanie 4: Estymacja współczynników równania regresji liniowej [8 pkt]

Przygotuj funkcję `interval_linear_regression_coefficients_estimation()`, która będzie dokonywała estymacji przedziałowej wartości współczynników równania regresji liniowej dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`a należy do przedziału [X1, Y1] przy założeniu poziomu ufności 1-𝛼 = Z1` </br>
`b należy do przedziału [X2, Y2] przy założeniu poziomu ufności 1-𝛼 = Z2`

gdzie X1, X2, Y1, Y2, Z1 i Z2 są automatycznie uzupełniane przez funkcję, a X1, X2, Y1 i Y2 dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [11]:
def linear_regression_coefficients_estimation(data, alpha):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i dwie kolumny "X" i "Y" z wynikami próby losowej.
    alpha: float
        Wartość określająca poziom istotności.
    Returns
    -------
    a_ci: tuple
        Zmienna typu tuple zawierajaca granice przedziału ufności parametru a (a_low, a_high).
    b_ci: tuple
        Zmienna typu tuple zawierajaca granice przedziału ufności parametru b (b_low, b_high).
    """   
    mean_x = data["X"].mean()
    mean_y = data["Y"].mean()
    length = len(data)

    a = np.sum(data["X"].values - mean_x) * np.sum(data["Y"].values - mean_y) / np.sum(data["X"].values - mean_x)
    b = mean_y - a * mean_x

    su_2 = 1 / (length - 2) * np.sum((data["Y"].values - (a * data["X"].values + b)) ** 2)
    D_a = su_2 ** (1/2) / np.sum((data["X"].values - mean_x) ** 2) ** (1/2)
    D_b = su_2 ** (1/2) * (np.sum(data["X"].values ** 2) / (length * np.sum((data["X"].values - mean_x) ** 2))) ** (1/2)

    a_lower = a - sp.stats.t.ppf(1 - alpha / 2, length - 2) * D_a
    a_higher = a + sp.stats.t.ppf(1 - alpha / 2, length - 2) * D_a

    b_lower = b - sp.stats.t.ppf(1 - alpha / 2, length - 2) * D_b
    b_higher = b + sp.stats.t.ppf(1 - alpha / 2, length - 2) * D_b

    return (a_lower, a_higher), (b_lower, b_higher)

linear_regression_coefficients_estimation(test_data_2, 0.1)

((-0.2146189264194453, 0.2146189264193032),
 (0.042323115792598176, 1.9386238929615227))

In [12]:
### Komórka testowa
assert np.all(np.isclose(linear_regression_coefficients_estimation(test_data_2, 0.1), ((1.90330932143819, 2.024457061652292), (-5.331039505544365, -4.7958291264947706))))
print()
assert np.all(np.isclose(linear_regression_coefficients_estimation(test_data_2, 0.05), ((1.8916219551002786, 2.0361444279902035), (-5.382672328053067, -4.744196303986069))))

AssertionError: 